In [ ]:
%load_ext blackcellmagic
# Usage: %%black

from dataclasses import dataclass, field
from typing import List
from queue import Queue, Empty
from random import shuffle, choice

import ipywidgets as widgets
from IPython.display import display, clear_output
import asyncio

# Enable widgets:
# !jupyter nbextension install --py widgetsnbextension --user
# !jupyter nbextension enable widgetsnbextension --user --py

In [ ]:
@dataclass
class Pairs:
    japanese: str
    english: str
    

@dataclass
class Player:
    name: str
    vocab: List[Pairs] = field(repr=False)
#     known: List[Pairs] = field(default_factory=lambda :[], init=False)
        
    def __post_init__(self):
        shuffle(self.vocab)
        
        self.to_check = Queue()
        for i in self.vocab:
            self.to_check.put(i)
            
        self.score = {
            "known": 0,
            "failed": 0,
            "skipped": 0,
        }
        
    def update(self, category):
        self.score[category] += 1
       
    def pick_word(self):
        choice = self.to_check.get_nowait()
        return choice
    
    def show_score(self):
        for k in sorted(self.score.keys()):
            print("{:<10}: {:>3}/{:>3} [{:>4.1%}]".format(
                k, self.score[k], len(self.vocab), self.score[k] / len(self.vocab)))


In [ ]:
_raw = """\
Healthy - けんこう
Calculate - けいさんする、計算する
Precise - せいかく、正確
Trick someone - だます、騙す
Free - むりょう, 無料
Trustable - しんよう (する), 信用【する】
A business - えいぎょう, 営業
As one would expect (/great) - さすが, 流石
International - こくさい, 国際
To entrust (to someone) - まかせる, 任せる
honest - しょうじき - 正直
feature - とくちょう - 特徴
basic - きほんてき - 基本的
to change - かえる - 変える
experiment - じっけん - 実験
efficiency - のうりつ - 能率
to improve - かいぜんする - 改善する
extreme - かげき - 過激
summary - おおすじ - 大筋
address - じゅうしょ - 住所
absence / non attendance - 欠席 / けっせき
rarely - めったに
to like / be pleased with - 気に入る / きにいる
grapes - ぶどう
sound / ring / roar - 鳴る / なる
board (board of wood) - 板 / いた
composition / essay - さくぶん　作文
surely certainly - きっと
to hold (hold the door) - 押さえる / おさえる
make a proposal / apply - 申し込む もうしこむ"""

pairs = [
    Pairs(english=eng, japanese=jap)
    for (eng, jap) in [x.split(" - ", maxsplit=1) for x in _raw.split("\n")]
]
# pairs

In [ ]:
%gui asyncio

def wait_for_change(widget, value):
    future = asyncio.Future()
    def getvalue(change):
        # TODO: learn how to use futures proprely
        try:
            future.result()
        except asyncio.InvalidStateError:
            future.set_result(0)
    widget.on_click(getvalue)
    return future

In [ ]:
_players = ["Dan", "Romeo", "Paul"]
players = [Player(name=name, vocab=pairs) for name in _players]
shuffle(players)
vocab_size = len(pairs) * len(players)

flags = {"japanese": "🇯🇵", "english": "🇬🇧", "aussie": "🇦🇺"}

question = widgets.HTML(
    value="Questions here!",
)

list_players = widgets.Select(
    options=[p.name for p in players],
    value=players[0].name,
    disabled=True
)

progress = widgets.IntProgress(
    value=0,
    min=0,
    max=vocab_size,
    step=1,
    description="[  0 /{:>3}]".format(vocab_size),
    bar_style='',
    orientation='horizontal'
)

status = widgets.ToggleButtons(
    options=['Known ', 'Unkown ', 'Skipped '],
    disabled=False,
    button_style='',
    icons=["check-circle", "question-circle", "forward"],
)
_mapping = {"Known ": "known", "Unkown ": "failed", "Skipped ": "skipped"}

next_b = widgets.Button(
    description='Next!',
    button_style='',
    icon='check'
)

async def f():
    for i in range(vocab_size):
        player_idx = i % len(players)
        curr_player = players[player_idx]
        list_players.value = curr_player.name
        word = curr_player.pick_word()

        q, a = choice([
            ["japanese", "english"],
            ["english", "japanese"],
        ])
        f_q, f_a = q, a
        if curr_player.name == "Dan":
            # Need the right flag
            f_q = "aussie" if q == "english" else f_q
            f_a = "aussie" if a == "english" else f_a
#         f_q = "aussie" if q == "english" and curr_player.name == "Dan" else q
#         f_a = "aussie" if q == "english" and curr_player.name == "Dan" else a

        question.value = "From <b>{}</b> {} to <b>{}</b> {}, translate:<p><b>{}</b>".format(
            q, flags[f_q], a, flags[f_a],
            getattr(word, q),
        )
        _ = await wait_for_change(next_b, 'value')
        curr_player.update(_mapping[status.value])
        
        progress.value += 1
        progress.description = "[{:>3} /{:>3}]".format(progress.value, vocab_size)
        
    print("All done")
    
    for p in players:
        print("~"*80)
        print(f"Name: {p.name}")
        p.show_score()

asyncio.ensure_future(f())

display(progress)
display(question)
display(list_players)
display(status)
next_b